In [1]:
#TO DO

#Create routine to aggregate data day by day in csv
#Aggregate by Heure de pointe du soir and Heure de pointe du matin Matin : 6h – 9h Soir : 16h – 19h
#Download in bulk data from septembre 2022 for comparison


import pandas as pd

# Load the CSV file into a DataFrame
# df = pd.read_csv(r'C:\Users\ambc\Documents\trains.csv', sep=',', )

df = pd.read_csv('https://raw.githubusercontent.com/amcaw/trains/main/data_trains.csv', sep=';',)
base = pd.read_csv('/Users/ambroisecarton/Downloads/operationele-punten-van-het-netwerk.csv', sep=';',)


In [2]:
# Remove rows where 'Direction de la relation' has no value (NaN)
df = df.dropna(subset=['Direction de la relation'])

# Select columns

df = df[['Date de départ', 'Numéro du train', 'Relation', 'Opérateur', 
                       'Heure réelle de départ', "Heure réelle d'arrivée", 'Retard au départ', 'Heure prévue de départ', "Direction de la relation", 'Nom du point d\'arrêt']]
# Select SNCB on

df = df[df['Opérateur'] == 'SNCB/NMBS']

In [3]:
# Cleaning
# Filter out rows where 'Heure réelle de départ' and 'Heure réelle d'arrivée' are the same so that we get rid of
# points d'arrêt that are not stations

df = df[df['Heure réelle de départ'] != df['Heure réelle d\'arrivée']]

# Remove rows where 'Relation' contains the values 'TGV' or 'ICE'
df = df[~df['Relation'].isin(['TGV', 'ICE'])]

# Extracting 'Départ de la ligne' and 'Arrivée de la ligne' from 'Direction de la relation' column
departure = df['Direction de la relation'].str.split('->').str[0].str.split(':').str[1].str.strip()
arrival = df['Direction de la relation'].str.split('->').str[1].str.strip()

# Updating the copied dataframe using .loc
df.loc[:, 'Départ de la ligne'] = departure
df.loc[:, 'Arrivée de la ligne'] = arrival

# Not so good but to translate in French

In [4]:

# Merge the 'df' dataframe with 'base' dataframe based on "Départ de la ligne" and "Abréviation LST NL complète"
merged_depart = pd.merge(df, base[['Abréviation LST NL complète', 'Nom FR complet']], 
                         left_on='Nom du point d\'arrêt', right_on='Abréviation LST NL complète', how='left')

# Replace the values in 'Départ de la ligne' with the corresponding 'Nom FR complet' values
merged_depart['Nom du point d\'arrêt'] = merged_depart['Nom FR complet'].fillna(merged_depart['Nom du point d\'arrêt'])

# Merge the updated 'merged_depart' dataframe with 'base' dataframe based on "Arrivée de la ligne" and "Abréviation LST NL complète"
merged_arrivee = pd.merge(merged_depart, base[['Abréviation LST NL complète', 'Nom FR complet']], 
                          left_on='Arrivée de la ligne', right_on='Abréviation LST NL complète', how='left', suffixes=('', '_arr'))

# Replace the values in 'Arrivée de la ligne' with the corresponding 'Nom FR complet_arr' values
merged_arrivee['Arrivée de la ligne'] = merged_arrivee['Nom FR complet_arr'].fillna(merged_arrivee['Arrivée de la ligne'])

# Drop the unnecessary columns from the merged dataframe
df = merged_arrivee.drop(columns=['Abréviation LST NL complète', 'Nom FR complet', 'Abréviation LST NL complète_arr', 'Nom FR complet_arr'])


In [5]:
#Drop columns qui sert plus à rien
df = df.drop(columns=['Heure réelle d\'arrivée', 'Direction de la relation', 'Départ de la ligne'])

In [6]:
# Select only weekdays from Monday to Friday

# Convert the "Date de départ" column to datetime format
df['Date de départ'] = pd.to_datetime(df['Date de départ'])

# Filter rows where the weekday is between Monday (0) and Friday (4)
df = df[df['Date de départ'].dt.weekday.between(0, 4)]

len(df)

78792

In [7]:
df = df.sort_values(by="Retard au départ", ascending=False)
df.head(10)

,Date de départ,Numéro du train,Relation,Opérateur,Heure réelle de départ,Retard au départ,Heure prévue de départ,Nom du point d'arrêt,Arrivée de la ligne
27079,2023-08-30,3716,IC 06-2,SNCB/NMBS,18:11:40,5500.0,16:40:00,Soignies,Brussels Airport - Zaventem
6150,2023-08-30,1737,IC 14,SNCB/NMBS,18:34:12,5472.0,17:03:00,Jurbise,Quiévrain
10149,2023-08-30,1716,IC 14,SNCB/NMBS,19:12:30,5370.0,17:43:00,Bruxelles-Central,Liège-Guillemins
5385,2023-08-30,1716,IC 14,SNCB/NMBS,19:07:28,5368.0,17:38:00,Bruxelles-Midi,Liège-Guillemins
27033,2023-08-30,3736,IC 06-2,SNCB/NMBS,18:02:33,5313.0,16:34:00,Jurbise,Mons
10150,2023-08-30,1716,IC 14,SNCB/NMBS,19:17:28,5308.0,17:49:00,Bruxelles-Nord,Liège-Guillemins
27080,2023-08-30,3716,IC 06-2,SNCB/NMBS,18:16:27,5247.0,16:49:00,Braine-le-Comte,Brussels Airport - Zaventem
10147,2023-08-30,1716,IC 14,SNCB/NMBS,18:33:58,5098.0,17:09:00,Soignies,Liège-Guillemins
10148,2023-08-30,1716,IC 14,SNCB/NMBS,18:49:49,5029.0,17:26:00,Hal,Liège-Guillemins
3307,2023-08-30,1716,IC 14,SNCB/NMBS,18:39:33,5013.0,17:16:00,Braine-le-Comte,Liège-Guillemins


In [8]:
# Convert 'Heure réelle de départ' to datetime format for filtering
#df['Heure réelle de départ'] = pd.to_datetime(df['Heure réelle de départ']).dt.time

# Filter the dataset for the time window between 6:00 AM and 9:00 AM
#time_filter = df[(df['Heure réelle de départ'] >= pd.to_datetime('06:00:00').time()) & 
                   #(df['Heure réelle de départ'] <= pd.to_datetime('09:00:00').time())]

# Group by the pair of columns "Nom FR complet" and "Arrivée de la ligne"
grouped = df.groupby(['Nom du point d\'arrêt', 'Arrivée de la ligne'])

In [9]:
# Calculate:
# 1. Count of trains for each group
train_count = grouped.size()

# 2. Total of minutes lost for each group
total_seconds_lost = grouped['Retard au départ'].sum()

# 3. Number of trains on time
on_time_count = grouped.apply(lambda x: (x['Retard au départ'] < 6*60).sum())


# 4. Number of trains late
late_count = grouped.apply(lambda x: (x['Retard au départ'] >= 6*60).sum())

# 5. Percentage of trains on time
on_time_percentage = ((on_time_count / train_count) * 100).round(2)

# Function to convert seconds to "X min Y s" format
def format_seconds_to_min_sec(seconds):
    minutes = int(seconds // 60)
    sec = int(seconds % 60)
    return f"{minutes} min {sec} s"

# Format the 'Total Minutes Lost' column
formatted_total_minutes_lost = total_seconds_lost.apply(format_seconds_to_min_sec)



# Create a summary dataframe
summary = pd.DataFrame({
    'Train Count': train_count,
    'Total Seconds Lost': total_seconds_lost,
    'Total Minutes Lost': formatted_total_minutes_lost,
    'On Time Count': on_time_count,
    'Late Count': late_count,
    'On Time Percentage (%)': on_time_percentage
}).reset_index()

summary

,Nom du point d'arrêt,Arrivée de la ligne,Train Count,Total Seconds Lost,Total Minutes Lost,On Time Count,Late Count,On Time Percentage (%)
0,Aalter,Blankenberge,34,8882.0,148 min 2 s,28,6,82.35
1,Aalter,Brussels Airport - Zaventem,34,3642.0,60 min 42 s,32,2,94.12
2,Aalter,Genk,36,3294.0,54 min 54 s,35,1,97.22
3,Aalter,Knokke,32,4194.0,69 min 54 s,29,3,90.62
4,Aalter,Malines,36,3737.0,62 min 17 s,34,2,94.44
...,...,...,...,...,...,...,...,...
2037,Zottegem,Termonde,36,10811.0,180 min 11 s,30,6,83.33
2038,Zottegem,Zottegem,96,0.0,0 min 0 s,0,0,0.00
2039,Zwijndrecht,Anvers-Central,64,5980.0,99 min 40 s,63,1,98.44
2040,Zwijndrecht,Lokeren,36,2294.0,38 min 14 s,36,0,100.00


In [10]:
summary = summary.sort_values(by="Total Seconds Lost", ascending=False)

summary.to_csv('export_trains.csv', index=False)

In [148]:
summary.head(10)

,Nom du point d'arrêt,Arrivée de la ligne,Train Count,Total Seconds Lost,Total Minutes Lost,On Time Count,Late Count,On Time Percentage (%)
57,Anvers-Berchem,Anvers-Central,253,35240.0,587 min 20 s,216,19,85.38
348,Bruxelles-Central,Bruxelles-Midi,104,22105.0,368 min 25 s,84,20,80.77
347,Bruxelles-Central,Brussels Airport - Zaventem,68,19283.0,321 min 23 s,53,15,77.94
1055,Jurbise,Quiévrain,17,18549.0,309 min 9 s,7,10,41.18
1883,Verviers-Palais,Spa-Géronstère,15,18110.0,301 min 50 s,0,15,0.00
1054,Jurbise,Mons,33,16949.0,282 min 29 s,24,9,72.73
1184,Lierre,Anvers-Central,99,16836.0,280 min 36 s,90,9,90.91
359,Bruxelles-Central,Liège-Guillemins,15,16142.0,269 min 2 s,7,8,46.67
450,Bruxelles-Nord,Liège-Guillemins,15,16034.0,267 min 14 s,7,8,46.67
401,Bruxelles-Midi,Brussels Airport - Zaventem,70,15686.0,261 min 26 s,56,13,80.00


In [149]:
# Filter the dataframe for trains from ANTWERPEN-CENTRAAL to HASSELT
antwerpen_to_hasselt_df = df[
    (df['Départ de la ligne'] == 'ANTWERPEN-CENTRAAL') &
    (df['Arrivée de la ligne'] == 'HASSELT')
]

antwerpen_to_hasselt_df

KeyError: 'Départ de la ligne'

In [ ]:
df = df[df['Numéro du train'] == 2606]
df

In [159]:
import requests
import pandas as pd
import os
from io import StringIO

# 1. Download the CSV
url = "https://opendata.infrabel.be/api/explore/v2.1/catalog/datasets/ruwe-gegevens-van-stiptheid-d-1/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
response = requests.get(url)
csv_content = response.text

# 2. Parse the CSV
new_data = pd.read_csv(StringIO(csv_content), delimiter=';')

# 3. Check if the CSV file exists. If not, create it. Otherwise, append the new data.
file_path = "data_trains.csv"

if os.path.exists(file_path):
    existing_data = pd.read_csv(file_path, delimiter=';')
    combined_data = pd.concat([existing_data, new_data], ignore_index=True)
    combined_data.to_csv(file_path, index=False, sep=';')
else:
    new_data.to_csv(file_path, index=False, sep=';')


In [29]:
df.to_csv('data_train_test.csv', index=False)